In [ ]:
# LangChain 101: from Zero to Hero

# Introduction

# Chapter 1: Installation and API keys

!pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken

import os
os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"

# Chapter 2: The LLMs

from langchain.llms import OpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the "OPENAI_API_KEY" environment variable.
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))

# Chapter 3: The Chains

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
   input_variables=["product"],
   template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))

# Chapter 4: The Memory

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
   llm=llm,
   verbose=True,
   memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)

# Chapter 5: Deep Lake VectorStore

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the "ACTIVELOOP_TOKEN" environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
   "Napoleon Bonaparte was born in 15 August 1769",
   "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "<YOUR-ACTIVELOOP-ORG-ID>"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

retrieval_qa = RetrievalQA.from_chain_type(
   llm=llm,
   chain_type="stuff",
   retriever=db.as_retriever()
)

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
   Tool(
       name="Retrieval QA System",
       func=retrieval_qa.run,
       description="Useful for answering questions."
   ),
]

agent = initialize_agent(
   tools,
   llm,
   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   verbose=True
)

response = agent.run("When was Napoleone born?")
print(response)

# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
   "Lady Gaga was born in 28 March 1986",
   "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

# instantiate the wrapper class for GPT3
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
   llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
   Tool(
       name="Retrieval QA System",
       func=retrieval_qa.run,
       description="Useful for answering questions."
   ),
]

# create an agent that uses the tool
agent = initialize_agent(
   tools,
   llm,
   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   verbose=True
)

response = agent.run("When was Michael Jordan born?")
print(response)

# Chapter 6: Agents in LangChain

from langchain.agents import load_tools
from langchain.utilities import GoogleSearchAPIWrapper

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# remember to set the environment variables
# "GOOGLE_API_KEY" and "GOOGLE_CSE_ID" to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
   Tool(
       name = "google-search",
       func=search.run,
       description="useful for when you need to search google to answer questions about current events"
   )
]

agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose=True,
                        max_iterations=6)

response = agent("What's the latest news about the Mars rover?")
print(response['output'])

# Chapter 7: Tools in LangChain

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

prompt = PromptTemplate(
   input_variables=["query"],
   template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

# remember to set the environment variables
# "GOOGLE_API_KEY" and "GOOGLE_CSE_ID" to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
   Tool(
       name="Search",
       func=search.run,
       description="useful for finding information about recent events"
   ),
   Tool(
      name='Summarizer',
      func=summarize_chain.run,
      description='useful for summarizing texts'
   )
]

agent = initialize_agent(
   tools,
   llm,
   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   verbose=True
)

response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])

# Conclusion